In [60]:
%pylab inline
%load_ext autoreload
%autoreload 2

import sys

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
sys.path.append('./ui')

In [78]:
from manager import Manager
from uitools import UITools
import gp

In [104]:
import mahotas as mh

In [68]:
E1_PATH = '/home/d/GPSTUDY/E1/'
E1_times = E1_PATH+'times.p'
E1_corrections = E1_PATH+'corrections.p'
E1_correction_vis = E1_PATH+'correction_vis.p'

import cPickle as pickle

with open(E1_times, 'rb') as f:
    E1_times = pickle.load(f)
with open(E1_corrections, 'rb') as f:
    E1_corrections = pickle.load(f)
with open(E1_correction_vis, 'rb') as f:
    E1_correction_vis = pickle.load(f)

# ORACLE_PATH = '/home/d/GPSTUDY/simuserGP_NEW/'
# ORACLE_corrections = ORACLE_PATH+'corrections.p'
# ORACLE_correction_vis = ORACLE_PATH+'correction_vis.p'

# with open(ORACLE_corrections, 'rb') as f:
#     ORACLE_corrections = pickle.load(f)
# with open(ORACLE_correction_vis, 'rb') as f:
#     ORACLE_correction_vis = pickle.load(f)    

In [105]:
OUTPUT_DIR = '/tmp/E1/'

In [136]:
html = '<tr>'
html += '<td class="number">{NUMBER}</td>'
html += '<td class="border {BEFORE_CSS}"><img src="gfx/{NUMBER}_border_before.png"</td>'
html += '<td class="border {AFTER_CSS}"><img src="gfx/{NUMBER}_border_after.png"</td>'
html += '<td class="border "><img src="gfx/{NUMBER}_gt.png"</td>'
html += '<td class="info">Oracle VI Delta: {DELTA_VI}<br>Label1 size: {LABEL1_SIZE}<br>Label2 size: {LABEL2_SIZE}</td>'
html += '</tr>'

In [137]:
output_html = ''

manager = Manager('/tmp/manager/')
manager.start('GP', 'nets/IPMLB_FULL.p', verbose=False)

error_count = 0
vi_count = 0
oracle_vi_count = 0

E1_old_vi = UITools.VI(manager._input_gold, manager._input_rhoana)[2]

merge_error = manager.get_next_merge_error()
while merge_error:
    
    #
    # grab prediction and images
    #
    prediction = merge_error[2]
    z = merge_error[0]
    border_before, border_after, labels_before, labels_after, slice_overview, cropped_slice_overview, bbox = manager.get_merge_error_image(merge_error, 0)
    
    gt = manager._input_gold[z][bbox[0]:bbox[1], bbox[2]:bbox[3]]
    
#    mh.imsave(OUTPUT_DIR+'gfx/'+str(error_count)+'_border_before.png', border_before)
#    mh.imsave(OUTPUT_DIR+'gfx/'+str(error_count)+'_border_after.png', border_after)
    mh.imsave(OUTPUT_DIR+'gfx/'+str(error_count)+'_gt.png', gp.Util.colorize(gt))

    
    
    #
    # get recorded E1 choice
    #
    clicked = E1_corrections[error_count][1]
    
    #
    # grab old VI from exactly this slice
    #
    old_vi = gp.Util.vi(manager._input_gold[z], manager._input_rhoana[z])
    
    #
    # correct the merge (only if the user selected a change)
    #
    mode, oracle_choice, oracle_delta_vi = manager.correct_merge(clicked, do_oracle=True)
    
    #
    # grab NEW VI from exactly this slice
    #
    new_vi = gp.Util.vi(manager._input_gold[z], manager._input_rhoana[z])
    
    #
    # VI DELTA
    #
    # should be positive
    delta_vi = old_vi - new_vi
    
    #print 'VI DELTA', delta_vi
    
    if clicked != oracle_choice:
        print 'ME', error_count, 'ORACLE DISAGREES', oracle_delta_vi
    else:
        print 'ME', error_count, 'ORACLE AGREES'
        
    before_css = ''
    after_css = ''
    if clicked == 'current':
        before_css += ' expert'
    else:
        after_css += ' expert'
        
    if oracle_choice == 'current':
        before_css += ' oracle'
    else:
        after_css += ' oracle'
        
        
    
    o_html = html.replace('{NUMBER}', str(error_count))
    o_html = o_html.replace('{BEFORE_CSS}', before_css)
    o_html = o_html.replace('{AFTER_CSS}', after_css)
    o_html = o_html.replace('{DELTA_VI}', str(max(0., oracle_delta_vi)))
    o_html = o_html.replace('{LABEL1_SIZE}', '?')
    o_html = o_html.replace('{LABEL2_SIZE}', '?') 
    output_html += o_html
        
    
    merge_error = manager.get_next_merge_error()
    error_count += 1
    
split_error = manager.get_next_split_error()
while split_error and error_count < len(E1_corrections):
    
    #
    # grab prediction and images
    #
    prediction = split_error[2]
    z = split_error[0]
    border_before, border_after, labels_before, labels_after, slice_overview, cropped_slice_overview, bbox = manager.get_split_error_image(split_error, 0)
    
    gt = manager._input_gold[z][bbox[0]:bbox[1], bbox[2]:bbox[3]]
    
#    mh.imsave(OUTPUT_DIR+'gfx/'+str(error_count)+'_border_before.png', border_before)
#    mh.imsave(OUTPUT_DIR+'gfx/'+str(error_count)+'_border_after.png', border_after)
    mh.imsave(OUTPUT_DIR+'gfx/'+str(error_count)+'_gt.png', gp.Util.colorize(gt))

    
    split_labels = split_error[1]
    label1_count = len(np.where(manager._input_rhoana==split_labels[0])[0])
    label2_count = len(np.where(manager._input_rhoana==split_labels[1])[0])    
    
    
    #
    # get recorded E1 choice
    #
    clicked = E1_corrections[error_count][1]
    
    #
    # grab old VI from exactly this slice
    #
    old_vi = gp.Util.vi(manager._input_gold[z], manager._input_rhoana[z])
    
    #
    # correct the merge (only if the user selected a change)
    #
    mode, oracle_choice, oracle_delta_vi = manager.correct_split(clicked, do_oracle=True)
    
    #
    # grab NEW VI from exactly this slice
    #
    new_vi = gp.Util.vi(manager._input_gold[z], manager._input_rhoana[z])
    
    #
    # VI DELTA
    #
    # should be positive
    delta_vi = old_vi - new_vi
    
    #print 'VI DELTA', delta_vi
    
    if clicked != oracle_choice:
        print 'SE', error_count, 'ORACLE DISAGREES', oracle_delta_vi, label1_count, label2_count
    else:
        print 'SE', error_count, 'ORACLE AGREES', label1_count, label2_count
    
    before_css = ''
    after_css = ''
    if clicked == 'current':
        before_css += ' expert'
    else:
        after_css += ' expert'
        
    if oracle_choice == 'current':
        before_css += ' oracle'
    else:
        after_css += ' oracle'
        
    
    o_html = html.replace('{NUMBER}', str(error_count))
    o_html = o_html.replace('{BEFORE_CSS}', before_css)
    o_html = o_html.replace('{AFTER_CSS}', after_css)
    o_html = o_html.replace('{DELTA_VI}', str(max(0., oracle_delta_vi)))
    o_html = o_html.replace('{LABEL1_SIZE}', str(label1_count))
    o_html = o_html.replace('{LABEL2_SIZE}', str(label2_count))
    output_html += o_html    
    
    split_error = manager.get_next_split_error()
    error_count += 1
    
    
    

ME 0 ORACLE AGREES
ME 1 ORACLE AGREES
ME 2 ORACLE DISAGREES 0.0
ME 3 ORACLE AGREES
ME 4 ORACLE AGREES
ME 5 ORACLE AGREES
ME 6 ORACLE AGREES
ME 7 ORACLE DISAGREES 0.0
ME 8 ORACLE AGREES
ME 9 ORACLE AGREES
SE 10 ORACLE AGREES 11311 28152
SE 11 ORACLE AGREES 6668 11367
SE 12 ORACLE AGREES 14425 28982
SE 13 ORACLE AGREES 7090 12120
SE 14 ORACLE AGREES 12199 10318
SE 15 ORACLE AGREES 10148 12503
SE 16 ORACLE AGREES 20902 17689
SE 17 ORACLE AGREES 11802 10207
SE 18 ORACLE AGREES 20090 17543
SE 19 ORACLE AGREES 19474 19753
SE 20 ORACLE AGREES 19941 23247
SE 21 ORACLE AGREES 17946 19286
SE 22 ORACLE AGREES 23481 12230
SE 23 ORACLE AGREES 17531 35960
SE 24 ORACLE AGREES 31378 22848
SE 25 ORACLE AGREES 27264 20235
SE 26 ORACLE AGREES 25066 19886
SE 27 ORACLE AGREES 35572 21105
SE 28 ORACLE AGREES 9211 10645
SE 29 ORACLE DISAGREES 0.00155217205887 33477 28412
SE 30 ORACLE AGREES 13590 11828
SE 31 ORACLE AGREES 22443 8714
SE 32 ORACLE AGREES 13590 7887
SE 33 ORACLE AGREES 39126 24170
SE 34 ORACLE 

In [138]:
html_header = '''<html>
<head>
<title>E1 GP performance</title>
<link rel="stylesheet" href="styles.css">
</head>


<body>
<font color='red'>RED</font>: EXPERT, <font color='blue'>BLUE</font>: ORACLE
<br><br>
<table width='100%' height='100%'>'''

html_footer = '''
</table>
</body>

</html>
'''


with open(OUTPUT_DIR+'/index.html', 'w') as f:
    
    f.write(html_header+output_html+html_footer)

In [96]:
a = manager._input_rhoana[0]

In [102]:
len(np.where(a==34)[0])

696

In [125]:
a = np.zeros((10,10))

In [128]:
a[0:5,0:5] = 1
a[0:7,7] = 2

In [127]:
len(np.where(a==1)[0])

25

In [129]:
len(np.where(a==2)[0])

7